# RNA-seq normalizations

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
path_counts = "/cellar/users/aklie/data/datasets/Hangauer2017_BT474_RNA-seq/results/quant/featureCounts/hangauer.results.counts"

# Load data

In [3]:
# Load counts
counts = pd.read_csv(path_counts, comment="#", sep="\t", index_col=0)
gene_info = counts.iloc[:, 0:5]
counts = counts.iloc[:, 5:]
counts.columns = [x.split("/")[-1].split(".")[0] for x in counts.columns]
counts.head()

,BT474_Parental_rep_1,BT474_Parental_rep_2,BT474_Persister_rep_1,BT474_Persister_rep_2,BT474_Persister_rep_3
Geneid,,,,,
ENSG00000223972.4,1,0,2,0,1
ENSG00000227232.4,48,58,121,114,151
ENSG00000243485.2,0,0,0,0,0
ENSG00000237613.2,0,0,0,0,0
ENSG00000268020.2,0,0,0,0,0


# CPM Normalization

In [4]:
from rnanorm import CPM

In [5]:
def normalize_cpm(counts_matrix):
    """
    Normalize an RNA-seq counts matrix using Counts Per Million (CPM).
    
    Counts Per Million (CPM) is a widely used method in RNA-seq data analysis to normalize 
    gene expression levels. CPM aims to adjust for differences in sequencing depths across 
    samples and provide relative expression values on a comparable scale by scaling the raw 
    read counts of each gene by a sample-specific sequencing depth (total counts) and 
    multiplying by a scaling factor of one million (to obtain counts per million).

    Pros:
    - Relatively simple and intuitive to implement.
    - Allows for direct comparisons of gene expression levels between samples.
    
    Cons:
    - Does not account for gene length, potentially leading to biases in gene expression estimation.
    - May be sensitive to extreme values or outliers.
    
    Parameters:
    counts_matrix (pd.DataFrame): A pandas DataFrame containing raw read counts 
                                  with rows as genes and columns as samples.
    
    Returns:
    pd.DataFrame: A pandas DataFrame with CPM-normalized expression values.
    
    Raises:
    ValueError: If the input is not a pandas DataFrame.
    """
    
    if not isinstance(counts_matrix, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame.")
    
    # Calculate total counts per sample
    total_counts = counts_matrix.sum(axis=0)
    
    # CPM normalization
    cpm_matrix = counts_matrix.div(total_counts, axis=1) * 1e6
    
    return cpm_matrix

In [6]:
# Normalize counts using CPM
cpm_counts = normalize_cpm(counts)
cpm_counts.head()

,BT474_Parental_rep_1,BT474_Parental_rep_2,BT474_Persister_rep_1,BT474_Persister_rep_2,BT474_Persister_rep_3
Geneid,,,,,
ENSG00000223972.4,0.008634,0.000000,0.018771,0.000000,0.009323
ENSG00000227232.4,0.414419,0.501441,1.135676,0.979852,1.407743
ENSG00000243485.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000268020.2,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
# use the rnanorm package to normalize the counts
CPM().set_output(transform="pandas").fit_transform(counts.T).T.head()

,BT474_Parental_rep_1,BT474_Parental_rep_2,BT474_Persister_rep_1,BT474_Persister_rep_2,BT474_Persister_rep_3
ENSG00000223972.4,0.008634,0.000000,0.018771,0.000000,0.009323
ENSG00000227232.4,0.414419,0.501441,1.135676,0.979852,1.407743
ENSG00000243485.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000268020.2,0.000000,0.000000,0.000000,0.000000,0.000000


# TPM Normalization

In [14]:
from rnanorm import TPM

In [15]:
def normalize_tpm(counts_matrix, gene_lengths):
    """
    Normalize an RNA-seq counts matrix using Transcripts Per Kilobase Million (TPM).
    
    Transcripts Per Kilobase Million (TPM) is an improvement over RPKM/FPKM. 
    TPM first normalizes for gene length, then for sequencing depth, making the sum 
    of all TPMs in each sample identical. This allows for a more accurate comparison 
    of gene expression between samples.
    
    Pros:
    - More accurate than RPKM/FPKM for comparing gene expression levels between samples.
    - Accounts for the total sum of normalized expression levels, allowing for a more balanced comparison.
    
    Cons:
    - Can be affected by highly expressed genes and depends on accurate estimates of gene lengths and accurate read mapping.
    - Cannot be used for differential expression analysis.
    
    Parameters:
    counts_matrix (pd.DataFrame): A pandas DataFrame containing raw read counts 
                                  with rows as genes and columns as samples.
    gene_lengths (pd.Series): A pandas Series containing the lengths of genes in kilobases.
                              The index should match the row index of the counts_matrix.
    
    Returns:
    pd.DataFrame: A pandas DataFrame with TPM-normalized expression values.
    
    Raises:
    ValueError: If the input counts_matrix is not a pandas DataFrame or if gene_lengths is not a pandas Series.
    """
    
    if not isinstance(counts_matrix, pd.DataFrame):
        raise ValueError("counts_matrix must be a pandas DataFrame.")
    
    if not isinstance(gene_lengths, pd.Series):
        raise ValueError("gene_lengths must be a pandas Series.")
    
    if not all(counts_matrix.index == gene_lengths.index):
        raise ValueError("The index of counts_matrix and gene_lengths must match.")
    
    # Normalize by gene length in kilobases
    length_normalized = counts_matrix.div(gene_lengths, axis=0)
    
    # Calculate per-sample scaling factors (total sum of length-normalized counts)
    scaling_factors = length_normalized.sum(axis=0)
    
    # Normalize by scaling factors and multiply by 1e6 to get TPM
    tpm_matrix = length_normalized.div(scaling_factors, axis=1) * 1e6
    
    return tpm_matrix

In [16]:
# Get gene lengths
gene_lengths = gene_info["Length"]
gene_lengths.head()

Geneid
ENSG00000223972.4    1756
ENSG00000227232.4    2073
ENSG00000243485.2    1021
ENSG00000237613.2    1219
ENSG00000268020.2     947
Name: Length, dtype: int64

In [17]:
# Normalize counts using TPM
tpm_counts = normalize_tpm(counts, gene_lengths)
tpm_counts.head()

,BT474_Parental_rep_1,BT474_Parental_rep_2,BT474_Persister_rep_1,BT474_Persister_rep_2,BT474_Persister_rep_3
Geneid,,,,,
ENSG00000223972.4,0.015742,0.000000,0.034341,0.000000,0.017247
ENSG00000227232.4,0.640070,0.784764,1.759937,1.536768,2.206111
ENSG00000243485.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000268020.2,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
# use the rnanorm package to normalize the counts
TPM(gene_lengths=gene_lengths).set_output(transform="pandas").fit_transform(counts.T).T.head()

,BT474_Parental_rep_1,BT474_Parental_rep_2,BT474_Persister_rep_1,BT474_Persister_rep_2,BT474_Persister_rep_3
ENSG00000223972.4,0.015742,0.000000,0.034341,0.000000,0.017247
ENSG00000227232.4,0.640070,0.784764,1.759937,1.536768,2.206111
ENSG00000243485.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000268020.2,0.000000,0.000000,0.000000,0.000000,0.000000


# RPKM/FPKM Normalization

In [23]:
from rnanorm import FPKM

In [27]:
import pandas as pd

def normalize_rpkm(counts_matrix, gene_lengths):
    """
    Normalize an RNA-seq counts matrix using Reads Per Kilobase of transcript per Million mapped reads (RPKM).
    
    RPKM (Reads Per Kilobase of transcript per Million mapped reads) and FPKM (Fragments Per Kilobase 
    of transcript per Million mapped reads) normalize for both the length of the gene and the total 
    number of reads (i.e., the library size), making expression level comparisons between genes 
    in the same sample possible.
    
    Pros:
    - Widely used and established/incorporated in several software tools.
    - Makes it possible to compare gene expression levels within the same sample and between different samples.
    
    Cons:
    - Assumes that the total number of reads is the same across all samples, which isn't always accurate, 
      particularly when comparing different conditions or tissues.
    - Can be biased by highly expressed genes or transcripts, making it less reliable for datasets 
      with a high level of expression variation.
    - Like TPM, it cannot be used for differential expression analysis.
    
    Parameters:
    counts_matrix (pd.DataFrame): A pandas DataFrame containing raw read counts 
                                  with rows as genes and columns as samples.
    gene_lengths (pd.Series): A pandas Series containing the lengths of genes in kilobases.
                              The index should match the row index of the counts_matrix.
    
    Returns:
    pd.DataFrame: A pandas DataFrame with RPKM-normalized expression values.
    
    Raises:
    ValueError: If the input counts_matrix is not a pandas DataFrame or if gene_lengths is not a pandas Series.
    """
    
    if not isinstance(counts_matrix, pd.DataFrame):
        raise ValueError("counts_matrix must be a pandas DataFrame.")
    
    if not isinstance(gene_lengths, pd.Series):
        raise ValueError("gene_lengths must be a pandas Series.")
    
    if not all(counts_matrix.index == gene_lengths.index):
        raise ValueError("The index of counts_matrix and gene_lengths must match.")
    
    # Divide by both gene length and total counts, then multiply by 1e9 to get RPKM
    rpkm_matrix = counts_matrix.div(gene_lengths, axis=0).div(counts_matrix.sum(axis=0), axis=1) * 1e9
    
    return rpkm_matrix

In [28]:
# Normalize counts using RPKM
rpkm_counts = normalize_rpkm(counts, gene_lengths)
rpkm_counts.head()

,BT474_Parental_rep_1,BT474_Parental_rep_2,BT474_Persister_rep_1,BT474_Persister_rep_2,BT474_Persister_rep_3
Geneid,,,,,
ENSG00000223972.4,0.004917,0.000000,0.010690,0.000000,0.005309
ENSG00000227232.4,0.199913,0.241892,0.547842,0.472673,0.679085
ENSG00000243485.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000268020.2,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
FPKM(gene_lengths=gene_lengths).set_output(transform="pandas").fit_transform(counts.T).T.head()

,BT474_Parental_rep_1,BT474_Parental_rep_2,BT474_Persister_rep_1,BT474_Persister_rep_2,BT474_Persister_rep_3
ENSG00000223972.4,0.004917,0.000000,0.010690,0.000000,0.005309
ENSG00000227232.4,0.199913,0.241892,0.547842,0.472673,0.679085
ENSG00000243485.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613.2,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000268020.2,0.000000,0.000000,0.000000,0.000000,0.000000
